# 目錄
* [A](#first-bullet)
* [所有處理函數](#second-bullet)

In [19]:
import glob
import os
from pathlib import Path
from PIL import Image
from PIL import ImageChops
import cv2 as cv
import numpy as np
from shutil import copyfile
from shutil import rmtree
from pathlib import Path
import ipywidgets
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from skimage import io
import time
#import "PaintingLight-v2.code.PaintingLight_save"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 設備檢查 
# Check enviroment

In [15]:
import os
#os.environ['Path']='C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.1\\bin;C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.1\\libnvvp;'+os.environ['Path']
#os.environ['Path']
import tensorflow as tf
#print(tf.__version__)

from tensorflow.python.client import device_lib
import tensorflow as tf
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
print(tf.__version__)
print(get_available_gpus())
!nvcc -V
device_lib.list_local_devices()

2.3.0
['/device:GPU:0']


'nvcc' 不是內部或外部命令、可執行的程式或批次檔。


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17236082472720559939,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8459070801696135077
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4932249856
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13796498536635575356
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:26:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14513591336033996820
 physical_device_desc: "device: XLA_GPU device"]

# 設定參數
# Setting parameters

In [ ]:
# for ShadeSketch
lightDirection1=210;
# for PaintingLight


# 所有處理函數
# All processing functions

In [20]:
#不做處理,直接複製
def copy_file(inPath,outPath):
    input_image=glob.glob(inPath+"/*")
    Path(outPath).mkdir(parents=True, exist_ok=True)
    for i in range(len(input_image)):  
        copyfile(input_image[i], outPath+"/"+str(i)+".png")
    return None
def delete_folder_content(path):
    rmtree(path)
    Path(path).mkdir(parents=True, exist_ok=True)
    #print(path+'\' content are deleted')
    return None
# 線稿修正1:line_thinning
def line_thinning(inPath,outPath):
    input_image=glob.glob(inPath+"/*")
    outpath=outPath
    Path(outpath).mkdir(parents=True, exist_ok=True)
    tags=[]
    for i in range(len(input_image)):
        outfile=os.path.join(outpath,str(i)+".png")
        !python line_thinning/thin.py {input_image[i]} {outfile}
        tags.append(str(i))
        #print("Line thining done, file at:"+outfile)
    #print("tags:"+str(tags))
    return None
# 線稿修正2:LineNormalizer
def LineNormalizer(inpath,outpath):
    delete_folder_content('LineNormalizer-master/input')
    delete_folder_content('LineNormalizer-master/output')
    #LineDistiller-master
    copy_file(inpath,'LineNormalizer-master/input')
    !python LineNormalizer-master/pytorch_predict_block.py
    copy_file('LineNormalizer-master/output',outpath)
    return None
# 線稿修正3:LineDistiller
def LineDistiller(inpath,outpath):
    delete_folder_content('LineDistiller-master/input')
    delete_folder_content('LineDistiller-master/output')
    #LineDistiller-master
    copy_file(inpath,'LineDistiller-master/input')
    !python LineDistiller-master/predict.py
    copy_file('LineDistiller-master/output',outpath)
    return None
# 線稿修正4:LineCloser
def LineCloser(inpath,outpath):
    delete_folder_content('LineCloser-master/input')
    delete_folder_content('LineCloser-master/output')
    #LineDistiller-master
    copy_file(inpath,'LineCloser-master/input')
    !python LineCloser-master/predict.py
    copy_file('LineCloser-master/output',outpath)
    return None
# 線稿修正5:LineReliver
def LineRelifer(inpath,outpath):
    delete_folder_content('LineRelifer-master/input')
    delete_folder_content('LineRelifer-master/output')
    #LineDistiller-master
    copy_file(inpath,'LineRelifer-master/input')
    !python LineRelifer-master/predict.py
    copy_file('LineRelifer-master/output',outpath)
    return None
# 線稿抽取:SketchKeras
def sketchKeras(inPath,outPath):
    !python sketchKeras-master/main.py --input_path {inPath} --output_path {outPath}
    img_list=glob.glob(outPath+'/*')
    for i in img_list:
        img = cv.imread(i)
        white_image=np.ones_like(img)*255
        white_image[3:-4,3:-4]=img[3:-4,3:-4]
        cv.imwrite(outPath+'/'+i.split('\\')[-1],white_image)
    return None
# Canny方法
def canny(inpath,outpath):
    img_list=glob.glob(inpath+'/*')
    Path(outpath).mkdir(parents=True, exist_ok=True)
    for i in img_list:
        
        img = cv.imread(i,0)
        edges = cv.Canny(img,100,20)
        edges=255-edges
        white_image=np.ones_like(edges)*255
        white_image[3:-4,3:-4]=edges[3:-4,3:-4]
        cv.imwrite(outpath+'/'+i.split('\\')[-1],white_image)
    return None
# 減去線條
def delete_Line(inpath,outpath):
    img_list=glob.glob(inpath+'/*')
    Path(outpath).mkdir(parents=True, exist_ok=True)
    for i in img_list:
        img = cv.imread(i,0)
        edges = cv.Canny(img,100,20)
        edges=255-edges
        white_image=np.ones_like(edges)*255
        white_image[3:-4,3:-4]=edges[3:-4,3:-4]
        cv.imwrite(outpath+'/'+i.split('\\')[-1],white_image)
# 上色GAN,標籤列表參考Tag2Pix/usedtags.csv¶
def coloring(inPath,outPath,tag):
    data_dir=Path(inPath).parent
    Path(outPath).mkdir(parents=True, exist_ok=True)
    tagspath = str(Path(inPath).parent)+'/tags.txt'
    f = open(tagspath, 'w')
    f.writelines(tag)
    f.close()
    !python Tag2Pix/main.py  --test --thread=1 --batch=1 --data_dir={data_dir} --tag_txt=tags.txt --test_dir={inPath.split("/")[-1]} --result_dir={outPath} --load=Tag2Pix/tag2pix_512.pkl --input_size=512 --no_bn
    return None
# 陰影GAN
def shadding(inPath,outPath):
    Path(outPath).mkdir(parents=True, exist_ok=True)
    !python ShadeSketch-master/predict.py --use-smooth --use-norm --direction 210 --input-dir {inPath} --output-dir {outPath}
    return None
# 把上色跟陰影組合
def combine(colorPath,shadingPath,outPath):
    #input_image=glob.glob(colorPath+"/tag2pix_512/*")
    input_image=glob.glob(colorPath+"/*")
    #print(input_image)
    input_image1=glob.glob(shadingPath+"/*")
    #print(input_image1)
    Path(outPath).mkdir(parents=True, exist_ok=True)
    for i in range(len(input_image)):
        color_img=Image.open(input_image[i])
        shading_img=Image.open(input_image1[i])
        #print(shading_img.size)
        #shading_img.show()
        color_img=color_img.resize(shading_img.size)
        #print(color_img.size)
        #color_img.show()
        output_img=ImageChops.multiply(color_img.convert('RGB'),shading_img.convert('RGB'))
        output_img.save(outPath+"/"+input_image[i].split('\\')[-1])
        #print(outPath+"/"+input_image[i].split('\\')[-1]+" is done")
    return None
#提取HSI線稿與明度
def HSI_Line(inpath,outpath_Line,outpath_Color,outpath_Lumin):
    Path(outpath_Color).mkdir(parents=True, exist_ok=True)
    Path(outpath_Color+"_RGB").mkdir(parents=True, exist_ok=True)
    Path(outpath_Line).mkdir(parents=True, exist_ok=True)
    Path(outpath_Lumin).mkdir(parents=True, exist_ok=True)
    all_Image=glob.glob(inpath+"/*")
    fig, axes = plt.subplots(ncols=3, figsize=(20, 20))
    plt.suptitle("orignal_Image",fontsize=50,y=0.7)
    for (index,i) in enumerate(all_Image):
        #print("processing:"+i)
        Image=io.imread(i)
        Image=cv.cvtColor(Image, cv.COLOR_RGB2HLS)
        gray=np.ones_like(Image[::,::,0])*128
        Image_Color=Image.copy()
        Image_Color[::,::,1]=gray
        Image_Line=Image.copy()
        Image_Line[::,::,0]=np.zeros_like(Image_Line[::,::,0])
        Image_Line[::,::,2]=np.zeros_like(Image_Line[::,::,2])
        Image_Line=cv.cvtColor(Image_Line, cv.COLOR_HLS2RGB)
        Image_Lumin=Image_Line.copy()
        
        Image_Line = cv.Canny(Image_Line,100,20)
        Image_Line=255-Image_Line
        (width,height)=(Image_Line.shape()[0],Image_Line.shape()[1])
        Image_Line=Image_Line[1:width-1,1,height-1]
        Image_Line=cv.resize(Image_Line,width,height)
        
        #ret,Image_Line=cv.threshold(Image_Line, 40, 255, cv.THRESH_BINARY)


        #print(Image_Line.shape)
        io.imsave(outpath_Color+"/"+str(index)+".jpg", Image_Color)
        
        Image_Color=cv.cvtColor(Image_Color, cv.COLOR_HLS2RGB)
        io.imsave(outpath_Color+"_RGB"+"/"+str(index)+".jpg", Image_Color)
        
        io.imsave(outpath_Lumin+"/"+str(index)+".jpg", Image_Lumin)
        io.imsave(outpath_Line+"/"+str(index)+".jpg", Image_Line)
        io.imshow(Image_Line)
    plt.show()
    return None
# HSI組合_改
def HSI_combine_kai(colorPath,luminPath,shadow_path,outPath):
    input_image=glob.glob(colorPath+"/*")
    input_image1=glob.glob(luminPath+"/*")
    input_image2=glob.glob(shadow_path+"/*")
    Path(outPath).mkdir(parents=True, exist_ok=True)
    for i in range(len(input_image)):
        
        lumin_img=Image.open(input_image1[i])
        shading_img=Image.open(input_image2[i])
        lumin=ImageChops.multiply(lumin_img.convert('RGB'),shading_img.convert('RGB'))
        lumin_image_final=np.array(lumin)
        lumin_image_final=cv.cvtColor(lumin_image_final, cv.COLOR_RGB2HLS)
        #print("lumin")
        #print(io.imshow(lumin_image_final))
        #plt.show()
        color_img=io.imread(input_image[i])
        color_img=cv.resize(color_img, lumin_img.size, interpolation=cv.INTER_AREA)
        #color_img=cv.cvtColor(color_img, cv.COLOR_RGB2HLS)
        #print("color")
        #print(io.imshow(color_img))
        #plt.show()
        color_img[::,::,1]=lumin_image_final[::,::,1]
        color_img=cv.cvtColor(color_img, cv.COLOR_HLS2RGB)
        io.imsave(outPath+"/"+input_image[i].split('\\')[-1],color_img)
        #print(outPath+"/"+input_image[i].split('\\')[-1]+" is done")
    return None
# HSI組合
def HSI_combine(colorPath,luminPath,outPath):
    input_image=glob.glob(colorPath+"/*")
    input_image1=glob.glob(luminPath+"/*")
    Path(outPath).mkdir(parents=True, exist_ok=True)
    for i in range(len(input_image)):
        
        lumin_img=io.imread(input_image1[i])
        #print(lumin_img)
        
        #lumin_img=cv.cvtColor(lumin_img, cv.COLOR_RGB2HLS)
        
        #print("lumin")
        #print(io.imshow(lumin_img))
        plt.show()
        color_img=io.imread(input_image[i])
        color_img=cv.resize(color_img,(lumin_img.shape[1],lumin_img.shape[0]),interpolation=cv.INTER_AREA)
        #print("color")
        #print(io.imshow(color_img))
        plt.show()
        color_img[::,::,1]=lumin_img
        color_img=cv.cvtColor(color_img, cv.COLOR_HLS2RGB)
        io.imsave(outPath+"/"+input_image[i].split('\\')[-1],color_img)
        #print(outPath+"/"+input_image[i].split('\\')[-1]+" is done")
    return None
# HSI組合_RGB
def HSI_combine_RGB(colorPath,luminPath,outPath):
    input_image=glob.glob(colorPath+"/*")
    input_image1=glob.glob(luminPath+"/*")
    Path(outPath).mkdir(parents=True, exist_ok=True)
    for i in range(len(input_image)):
        
        lumin_img=io.imread(input_image1[i])
        #print(lumin_img)
        
        #lumin_img=cv.cvtColor(lumin_img, cv.COLOR_RGB2HLS)
        
        #print("lumin")
        #print(io.imshow(lumin_img))
        plt.show()
        color_img=io.imread(input_image[i])
        color_img=cv.resize(color_img,(lumin_img.shape[1],lumin_img.shape[0]),interpolation=cv.INTER_AREA)
        #print("color")
        #print(io.imshow(color_img))
        plt.show()
        color_img=cv.cvtColor(color_img, cv.COLOR_RGB2HLS)
        color_img[::,::,1]=lumin_img
        color_img=cv.cvtColor(color_img, cv.COLOR_HLS2RGB)
        io.imsave(outPath+"/"+input_image[i].split('\\')[-1],color_img)
        #print(outPath+"/"+input_image[i].split('\\')[-1]+" is done")
    return None
def paintingLight_process(inpath,outpath, mask, ambient_intensity, light_intensity, light_source_height,
            gamma_correction, stroke_density_clipping, light_color_red, light_color_green,
            light_color_blue, enabling_multiple_channel_effects,light_position=[-0.8,0.8]):
    img_list=glob.glob(inpath+"/*")
    Path(outpath).mkdir(parents=True, exist_ok=True)
    for i in img_list:
        image = cv.imread(i)
        output_name=i.split("\\")[-1]
        #light_source_height =2
        #gamma_correction = 1.0
        #stroke_density_clipping = 0.5
        #enabling_multiple_channel_effects = True
        #light_intensity=1
        #ambient_intensity=0
        #light_color_red = 1.0
        #light_color_green = 1.0
        #light_color_blue = 1.0
        (gx,gy)=(light_position[0],light_position[1])
        #(gx,gy)在畫面上的座標
        #(1,1)---------- (-1,1)
        #|                |
        #|       (0,0)    |
        #|                |
        #(1,-1)----------(-1,-1)
        
        paintingLight_run(image, mask, ambient_intensity, light_intensity, light_source_height,
            gamma_correction, stroke_density_clipping, light_color_red, light_color_green,
            light_color_blue, enabling_multiple_channel_effects,gx,gy,outpath+"/"+output_name)

# 流程1

In [24]:
startTime=time.perf_counter()
canny("test_image","test_output/tmp/canny_Line")
shadding("test_output/tmp/canny_Line","test_output/tmp/shade_canny")
combine("test_image","test_output/tmp/shade_canny","test_output/process_1")
endTime=time.perf_counter()
print("運算時間為:",endTime-startTime,"秒") 

Running inference for test_output/tmp/canny_Line\whatever.jpg ...
運算時間為: 11.081167499999992 秒


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use tf.gfile.GFile.


# 流程2

In [ ]:
startTime=time.perf_counter()
sketchKeras("test_image","test_output/tmp/sketch_Keras")
shadding("test_output/tmp/sketch_Keras","test_output/tmp/shade_sketch_Keras")
combine("test_image","test_output/tmp/shade_sketch_Keras","test_output/process_2")
endTime=time.perf_counter()
print("運算時間為:",endTime-startTime,"秒") 

## 流程3(棄用)

In [ ]:

LineCloser("test_output/tmp/canny_Line","test_output/tmp/canny_Line_Closer")

LineRelifer("test_output/tmp/canny_Line_Closer","test_output/tmp/canny_Line_Closer_Relifer")

shadding("test_output/tmp/canny_Line_Closer_Relifer","test_output/tmp/shadding_canny_Line_Closer_Relifer")

combine("test_image","test_output/tmp/shadding_canny_Line_Closer_Relifer","test_output/process_4")

In [ ]:
sketchKeras("test_output/tmp/canny_Line","test_output/tmp/canny_sketch_Keras")
shadding("test_output/tmp/canny_sketch_Keras","test_output/tmp/shade_canny_sketch_Keras")
combine("test_image","test_output/tmp/shade_canny_sketch_Keras","test_output/tmp/combine_process_3")

## 流程3(PaintingLight)

In [25]:
startTime=time.perf_counter()
mask = None
light_source_height =1
gamma_correction =1
stroke_density_clipping = 1
enabling_multiple_channel_effects = True
light_intensity=1
ambient_intensity=0.7
light_color_red = 1.0
light_color_green = 1.0
light_color_blue = 1.0
Light_direction=[-0.8,0.8]
paintingLight_process("test_output/process_2","test_output/final", mask, ambient_intensity, light_intensity, light_source_height,gamma_correction, stroke_density_clipping, light_color_red, light_color_green,light_color_blue, enabling_multiple_channel_effects,Light_direction)
endTime=time.perf_counter()
print("運算時間為:",endTime-startTime,"秒") 


運算時間為: 0.0008697000002939603 秒


## 各種比較圖 ##

In [ ]:

(img_list1,img_list2,img_list3)=(glob.glob("test_output/process_1/*"),glob.glob("test_output/process_2/*"),glob.glob("test_output/process_3/*"))
Path("test_output/compare").mkdir(parents=True, exist_ok=True)
for i in range(len(img_list1)):
    figure(figsize=(20,16), dpi=80)
    plt.title(img_list1[i].split('\\')[-1]), plt.xticks([]), plt.yticks([])
    img=cv.imread(img_list1[i])
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(1,3,1),plt.imshow(img)
    img=cv.imread(img_list2[i])
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(1,3,2),plt.imshow(img)
    img=cv.imread(img_list3[i])
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(1,3,3),plt.imshow(img)
    
    plt.savefig("test_output/compare/"+img_list1[i].split("\\")[-1])
    
        

In [ ]:

(img_list1,img_list2)=(glob.glob("test_output/tmp/canny_Line/*"),glob.glob("test_output/tmp/sketch_Keras/*"))
Path("test_output/compare_Line").mkdir(parents=True, exist_ok=True)
for i in range(len(img_list1)):
    figure(figsize=(20,16), dpi=80)
    plt.title(img_list1[i].split('\\')[-1]), plt.xticks([]), plt.yticks([])
    img=cv.imread(img_list1[i])
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(1,2,1),plt.imshow(img)
    img=cv.imread(img_list2[i])
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(1,2,2),plt.imshow(img)
    
    plt.savefig("test_output/compare_Line/"+img_list1[i].split("\\")[-1])
    
    #plt.show()
    
        

In [ ]:

(img_list1,img_list2)=(glob.glob("test_output/tmp/shade_canny/*"),glob.glob("test_output/tmp/shade_sketch_Keras/*"))
Path("test_output/compare_Shadow").mkdir(parents=True, exist_ok=True)
for i in range(len(img_list1)):
    figure(figsize=(20,16), dpi=80)
    plt.title(img_list1[i].split('\\')[-1]), plt.xticks([]), plt.yticks([])
    img=cv.imread(img_list1[i])
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(1,2,1),plt.imshow(img)
    img=cv.imread(img_list2[i])
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(1,2,2),plt.imshow(img)
    plt.savefig("test_output/compare_Shadow/"+img_list1[i].split("\\")[-1])
    
    #plt.show()

In [ ]:
combine("test_output/tmp/shade_canny","test_output/tmp/canny_Line","test_output/tmp/combine_process_1_BW")

combine("test_output/tmp/shade_sketch_Keras","test_output/tmp/sketch_Keras","test_output/tmp/combine_process_2_BW")

(img_list1,img_list2)=(glob.glob("test_output/tmp/combine_process_1_BW/*"),glob.glob("test_output/tmp/combine_process_2_BW/*"))
Path("test_output/compare_BW").mkdir(parents=True, exist_ok=True)
for i in range(len(img_list1)):
    figure(figsize=(20,16), dpi=80)
    plt.title(img_list1[i].split('\\')[-1]), plt.xticks([]), plt.yticks([])
    img=cv.imread(img_list1[i])
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(1,2,1),plt.imshow(img)
    img=cv.imread(img_list2[i])
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(1,2,2),plt.imshow(img)
    
    plt.savefig("test_output/compare_BW/"+img_list1[i].split("\\")[-1])

## 陰影抽取

In [ ]:
shadding("test_image\inked","test_output_inked")